In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import os
import sys
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.activations import *
import matplotlib.pyplot as plt
from IPython import *
import cv2
from tqdm import *
import pickle
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import *
from drive.MyDrive.BTP.utils import *

In [3]:
var_dtype='float32'

In [4]:
#tk.backend.clear_session()
tk.backend.set_floatx(var_dtype)

In [5]:
#q1_im1
#q1_op1
def im(q,ind,ext=".JPG"):
    return "q"+str(q)+"_im"+str(ind)+ext

def op(q,ind,ext=".JPG"):
    return "q"+str(q)+"_op"+str(ind)+ext

def getNames(q):
    return [im(q,i) for i in range(1,5)]+[op(q,i) for i in range(1,5)]


def imshow(img):
    plt.imshow(img.astype(np.uint8))
    plt.show()

def imread(fname,target_shape=None):
    img=cv2.imread(fname)
    if target_shape is not None: return cv2.resize(img,target_shape,interpolation=cv2.INTER_AREA)
    return img

def resize(img,target_shape):
    return cv2.resize(img,target_shape,interpolation=cv2.INTER_AREA)

In [6]:
def generator(x,bsize=32):
    if bsize==-1: yield x
    i=0
    n=x.shape[0]
    while i+bsize<=n:
        yield x[i:i+bsize]
        i+=bsize
    if i<n: yield x[i:]


def generator1(x,y,bsize=32):
    if bsize==-1: yield x,y
    i=0
    n=x.shape[0]
    while i+bsize<=n:
        yield x[i:i+bsize],y[i:i+bsize]
        i+=bsize
    if i<n: yield x[i:],y[i:]

In [7]:
def convert2vector(imgs,encoder):
    assert len(imgs.shape)==5
    r=[]
    n=imgs.shape[0]
    for i in range(n):
        r.append(encoder(imgs[i]).numpy())
    return np.array(r,dtype=var_dtype)

In [8]:
root_dir="/content/drive/MyDrive/BTP/"
root_data_dir=root_dir+"created_dataset/"
csv=root_dir+"image_cop.csv"

encoder_weights=root_dir+"trained_models/encoder.h5"
rel_decoder_weights=root_dir+"trained_models/rel_decoder.h5"

classifier_weights=root_dir+"trained_models/classifier.h5"
lstm_weights=root_dir+"trained_models/lstm.pkl"

inp_image_size=(64,64)
inp_image_channel=out_image_channel=3
inp_image_shape=inp_image_size+(inp_image_channel,)
batch_size=32
#rel_decoder_input_shape=(6,6,2048)

initilizer=tk.initializers.RandomUniform(minval=-0.1, maxval=0.1, seed=None)

In [9]:
load_encoder_weights=root_dir+"backup/encoder-latest.h5"
load_rel_decoder_weights=root_dir+"backup/rel_decoder-latest.h5"

#load_classifier_weights=root_dir+"backup/classifier-full.h5"
load_lstm_weights=root_dir+"backup/lstm-latest.pkl"

In [ ]:
# q=0
# x_train=[]
# y_train=[]
# df=pd.read_csv(csv)
# for _ in tqdm(range(len(df))):
#     names=getNames(q)
#     try:
#         t=[]
#         for name in names:
#             path=root_data_dir+name
#             t.append(imread(path,inp_image_size))
        
#         x_train.append(t)
    
#     except Exception as e:
#         print(e)
#         break
    
#     q+=1

In [ ]:
# x_train=np.array(x_train).astype(var_dtype)
# np.save(root_dir+'images_arrays.npy',x_train,allow_pickle=True)
# x_train.shape

In [ ]:
# train=np.load(root_dir+'images_arrays.npy').astype(var_dtype)/255.0
# train,test=train_test_split(train,test_size=0.3)
# np.save(root_dir+'train_images_arrays.npy',train,allow_pickle=True)
# np.save(root_dir+'test_images_arrays.npy',test,allow_pickle=True)

In [ ]:
train_imgs=train=np.load(root_dir+'train_images_arrays.npy').astype(var_dtype)
test_imgs=test=np.load(root_dir+'test_images_arrays.npy').astype(var_dtype)
imshow(train_imgs[0][0]*255)

In [11]:
#test=np.concatenate([test,np.load(root_dir+'test_images_arrays.npy').astype(var_dtype)/255.0],axis=0)

In [12]:
encoder=getEncoder(trainable=False,weights=load_encoder_weights)#load_encoder_weights
encoder.trainable=True

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
#train=convert2vector(train,encoder)
#test=convert2vector(test,encoder)

In [13]:
train.shape,test.shape

((1050, 8, 64, 64, 3), (450, 8, 64, 64, 3))

In [14]:
recurrent_activation="sigmoid"
insert_time_axis=Lambda(lambda x: tf.expand_dims(x,axis=1))
lstm=LSTM(128,activation=None,recurrent_activation=recurrent_activation,return_sequences=True,return_state=True,input_shape=(1,)+(encoder.output_shape[1:]))#,kernel_initializer=initilizer,bias_initializer='zeros')#recurrent_activation=LeakyReLU(),

In [15]:
x,y,z=lstm(tf.ones(shape=(1,1)+(encoder.output_shape[1:]),dtype=var_dtype))

In [16]:
with open(load_lstm_weights, 'rb') as f:
   lstm.set_weights(pickle.load(f))

In [17]:
lstm.units

128

In [ ]:
# classifier_inp=Input(shape=(lstm.units,))
# x=Dense(128,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(classifier_inp)
# x=Dense(64,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(x)
# x=Dense(32,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(x)
# #x=Dense(512,activation='relu',kernel_initializer=initilizer,bias_initializer='zeros')(x)
# classifier_out=Dense(1,activation='sigmoid',kernel_initializer=initilizer,bias_initializer='zeros')(x)
# classifier=tk.Model(inputs=classifier_inp,outputs=classifier_out)

In [ ]:
#classifier.load_weights(load_classifier_weights)

In [18]:
rel_decoder=getDecoder(encoder,lstm_units=lstm.units)
rel_decoder.trainable=True
#plot_model(rel_decoder,show_shapes=True)

In [19]:
rel_decoder.load_weights(load_rel_decoder_weights)

In [20]:
def MSE(true,pred):
    #true=tf.reshape(true,(-1,1))
    #pred=tf.reshape(pred,(-1,1))
    err=tf.square(true-pred)
    err=tf.reduce_mean(err)
    return err

def reconstruction_loss(x_train,y_train,relation):
    assert len(x_train.shape)==2
    assert x_train.shape[0]==8

    n=1
    for i in x_train.shape[:-1]:
        n=n*i
    
    zeros=tf.zeros(shape=(n,)+rel_decoder.input_shape[0][1:],dtype=var_dtype)
    img_vectors=x_train#.reshape(-1,x_train.shape[-1])
    gen_imgs=rel_decoder([zeros,img_vectors])
    mse_error=MSE(y_train*255,gen_imgs)#.reshape(y_train.shape[0]*y_train.shape[1],)+inp_image_shape)

    decode_error=0.0
    for i in range(4):
        decode_error+=MSE(y_train[i+1:i+2]*255,rel_decoder([relation,x_train[i:i+1]]))/4

    #print(mse_error,decode_error)

    return mse_error+decode_error

In [21]:
train_imgs.shape,train.shape,rel_decoder.input_shape

((1050, 8, 64, 64, 3), (1050, 8, 64, 64, 3), [(None, 128), (None, 2048)])

In [49]:
def train_loop(x_train,y_train,opt,update_weights=True,train_encoder=False,train_decoder=True,print_pred=False):#,apply_recons=True
    n=x_train.shape[0]
    correct_pred=0.0

    if var_dtype=='float32': n=np.float32(n)
    else: n=np.float64(n)

    with tf.GradientTape(persistent=True) as tape:
        cls_loss=0.0
        recons_loss=0.0
        for i in range(x_train.shape[0]):
            if train_encoder: encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4=tf.expand_dims(encoder(x_train[i]),axis=1)
            else: encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4=tf.expand_dims(x_train[i],axis=1)
            #print(encoded_img1.shape)

            encoded_vec=tf.concat([encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4],axis=0)

            out,h_t1,c_t1=lstm(insert_time_axis(encoded_img1))
            out,h_t2,c_t2=lstm(insert_time_axis(encoded_img2),initial_state=[h_t1,c_t1])
            out,h_t3,c_t3=lstm(insert_time_axis(encoded_img3),initial_state=[h_t2,c_t2])
            out,h_t4,c_t4=lstm(insert_time_axis(encoded_img4),initial_state=[h_t3,c_t3])


            #out_op1,h_t5_op1,c_t5_op1=lstm(insert_time_axis(encoded_op1),initial_state=[h_t4,c_t4])
            #out_op2,h_t5_op2,c_t5_op2=lstm(insert_time_axis(encoded_op2),initial_state=[h_t4,c_t4])
            #out_op3,h_t5_op3,c_t5_op3=lstm(insert_time_axis(encoded_op3),initial_state=[h_t4,c_t4])
            #out_op4,h_t5_op4,c_t5_op4=lstm(insert_time_axis(encoded_op4),initial_state=[h_t4,c_t4])

            #classification_lable_op1=classifier(h_t5_op1)
            #classification_lable_op2=classifier(h_t5_op2)
            #classification_lable_op3=classifier(h_t5_op3)
            #classification_lable_op4=classifier(h_t5_op4)

            relation=h_t4

            if train_decoder: recons_loss+=reconstruction_loss(encoded_vec,y_train[i],relation)/n


            #shape=classification_lable_op1.shape
            #dtype=classification_lable_op1.dtype

            
            #pred=tf.concat([classification_lable_op1,classification_lable_op2,classification_lable_op3,classification_lable_op4],axis=0)
            #if print_pred: print(pred)
            #true=tf.concat([tf.ones(shape,dtype),tf.zeros(shape,dtype),tf.zeros(shape,dtype),tf.zeros(shape,dtype)],axis=0)


            
            #cls_loss+=(binary_crossentropy(true,pred))


        #cls_loss=cls_loss[0]*3.0+cls_loss[1]+cls_loss[2]+cls_loss[3]#/x_train.shape[0]#+tf.reshape(recons_loss,cls_loss.shape)
        total_loss=1000*cls_loss+recons_loss
    
    print("Error :",total_loss)
    print("Cls Loss :",cls_loss)
    print("Recons Loss :",recons_loss)
    
    if update_weights:
        #print("Classification Loss :",cls_loss,"\n")
        #print("Reconstruction Loss :",recons_loss,"\n")

        #opt.learning_rate=1e-3
        lstm_gradients=tape.gradient(total_loss,lstm.trainable_variables)
        opt.apply_gradients(zip(lstm_gradients,lstm.trainable_variables))

        #opt.learning_rate=1e-4
        #classifier_gradients=tape.gradient(total_loss,classifier.trainable_variables)
        #opt.apply_gradients(zip(classifier_gradients,classifier.trainable_variables))
        
        #opt.learning_rate=1e-3
        if train_encoder:
            encoder_gradients=tape.gradient(total_loss,encoder.trainable_variables)
            opt.apply_gradients(zip(encoder_gradients,encoder.trainable_variables))
    
        #opt.learning_rate=1e-3
        if train_decoder:
            rel_decoder_gradients=tape.gradient(total_loss,rel_decoder.trainable_variables)
            opt.apply_gradients(zip(rel_decoder_gradients,rel_decoder.trainable_variables))

    return total_loss.numpy()

In [23]:
opt=Adam(1e-6)

In [ ]:
tk.backend.set_value(opt.learning_rate,2e-4)
for i in tqdm(range(100)):
    epochs=1
    indx=random.sample(range(train.shape[0]),32)
    for _ in range(epochs):
        print("\nEpoch {}:".format(_+1))
        loss=train_loop(train[indx],train_imgs[indx],opt,train_encoder=True,train_decoder=True)
        if loss>4000: loss=train_loop(train[indx],train_imgs[indx],opt,train_encoder=True,train_decoder=True)
    display.clear_output()

In [ ]:
encoder.save_weights(encoder_weights)
#classifier.save_weights(classifier_weights)
with open(lstm_weights,'wb') as f:
    pickle.dump(lstm.get_weights(),f)
rel_decoder.save_weights(rel_decoder_weights)

In [40]:
def gen_next_image(x_train):
    encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4=tf.expand_dims(encoder(x_train),axis=1)
    #encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4=tf.expand_dims(x_train,axis=1)

    out,h_t1,c_t1=lstm(insert_time_axis(encoded_img1))
    out,h_t2,c_t2=lstm(insert_time_axis(encoded_img2),initial_state=[h_t1,c_t1])
    out,h_t3,c_t3=lstm(insert_time_axis(encoded_img3),initial_state=[h_t2,c_t2])
    out,h_t4,c_t4=lstm(insert_time_axis(encoded_img4),initial_state=[h_t3,c_t3])


    #out_op1,h_t5_op1,c_t5_op1=lstm(insert_time_axis(encoded_op1),initial_state=[h_t4,c_t4])
    #out_op2,h_t5_op2,c_t5_op2=lstm(insert_time_axis(encoded_op2),initial_state=[h_t4,c_t4])
    #out_op3,h_t5_op3,c_t5_op3=lstm(insert_time_axis(encoded_op3),initial_state=[h_t4,c_t4])
    #out_op4,h_t5_op4,c_t5_op4=lstm(insert_time_axis(encoded_op4),initial_state=[h_t4,c_t4])

    relation=h_t4
            
    gen=rel_decoder([relation,encoded_img4]).numpy()[0]
    gen=np.clip(gen,0,255)
    imshow(x_train[4]*255)
    imshow(gen)

In [ ]:
gen_next_image(train_imgs[10])
gen_next_image(train_imgs[500])
gen_next_image(test_imgs[10])
gen_next_image(test_imgs[300])

In [ ]:
def test_accuracy(x_train,encode=False,print_pred=False):
    total_len=(x_train.shape[0])*4
    correct_pred=0.0

    if var_dtype=='float32': total_len=np.float32(total_len)
    else: total_len=np.float64(total_len)

    for i in range(x_train.shape[0]):
        if encode: encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4=tf.expand_dims(encoder(x_train[i]),axis=1)
        else: encoded_img1,encoded_img2,encoded_img3,encoded_img4,encoded_op1,encoded_op2,encoded_op3,encoded_op4=tf.expand_dims(x_train[i],axis=1)

        out,h_t1,c_t1=lstm(insert_time_axis(encoded_img1))
        out,h_t2,c_t2=lstm(insert_time_axis(encoded_img2),initial_state=[h_t1,c_t1])
        out,h_t3,c_t3=lstm(insert_time_axis(encoded_img3),initial_state=[h_t2,c_t2])
        out,h_t4,c_t4=lstm(insert_time_axis(encoded_img4),initial_state=[h_t3,c_t3])


        out_op1,h_t5_op1,c_t5_op1=lstm(insert_time_axis(encoded_op1),initial_state=[h_t4,c_t4])
        out_op2,h_t5_op2,c_t5_op2=lstm(insert_time_axis(encoded_op2),initial_state=[h_t4,c_t4])
        out_op3,h_t5_op3,c_t5_op3=lstm(insert_time_axis(encoded_op3),initial_state=[h_t4,c_t4])
        out_op4,h_t5_op4,c_t5_op4=lstm(insert_time_axis(encoded_op4),initial_state=[h_t4,c_t4])

        classification_lable_op1=classifier(h_t5_op1)
        classification_lable_op2=classifier(h_t5_op2)
        classification_lable_op3=classifier(h_t5_op3)
        classification_lable_op4=classifier(h_t5_op4)

        #relation=h_t4
            
        pred=tf.concat([classification_lable_op1,classification_lable_op2,classification_lable_op3,classification_lable_op4],axis=0)
        
        if print_pred: print(pred)

        if(pred[0][0]>pred[1][0] and pred[0][0]>pred[2][0] and pred[0][0]>pred[3][0]): correct_pred+=1


    print("Accuracy :",(correct_pred/x_train.shape[0]))

In [ ]:
test_accuracy(test,print_pred=False)

Accuracy : 0.7777777777777778


In [ ]:
test_accuracy(train,print_pred=False)

Accuracy : 0.9342857142857143


In [ ]:
test_accuracy(test[0:10],print_pred=True)